# Pre-processing

In [1]:
import pandas as pd
import numpy as np

In [115]:
coding=pd.read_csv("coding.csv",header=0,index_col=0)

In [14]:
likeall=pd.read_csv("./commall.csv",header=None, names=['time','pageid','uid'])

In [23]:
sample=likeall[likeall['uid'].isin(coding['fbid'])]

In [30]:
sample["count"]=1

C:\Users\junior\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [32]:
sample_matrix=sample.pivot_table(index='uid',columns='pageid',values='count',aggfunc=np.sum)

In [46]:
sample_matrix=sample_matrix.replace("NaN",0)

In [47]:
sample_matrix

pageid,8100113319,8748127833,10795878252,12840197743,13716140500,18809502474,27743445015,28211179048,46251501064,46446727755,...,1524212551168001,1533327983576847,1545691905710112,1546925482228025,1559300274285980,1562471480709108,1567610543464836,1600171100224538,1631420700428579,1638125039738820
uid,,,,,,,,,,,,,,,,,,,,,
4761454131026,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
305862886286264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315729135278571,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
345972642250855,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
347478358766645,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
399526776865260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
462711943866612,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
495394180603827,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
500169920085926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0


# NMF

In [34]:
from sklearn.decomposition import NMF

C:\Users\junior\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\junior\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [417]:
model = NMF(n_components=30, init='random',alpha=0.01,max_iter=500,tol=1e-4,random_state=0)
sample_nmf=model.fit_transform(sample_matrix)

In [408]:
#component-page
model.components_.shape

(80L, 343L)

In [98]:
#user-component
sample_nmf.shape

(150L, 50L)

In [418]:
#(V-WH)**2
model.reconstruction_err_

24.650661569383963

# Training Models

## Split Dataset

In [398]:
result=coding.sort_index(axis=0, ascending=1)['yellow']
result=result.reset_index()
del result['fbid']

In [399]:
from sklearn import cross_validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(sample_nmf, result, test_size=0.3)

## Logistic

In [400]:
from sklearn.linear_model import LogisticRegression
LogReg=LogisticRegression()
LogReg.fit(X_train, y_train)

C:\Users\junior\Anaconda2\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [396]:
print LogReg.predict(X_train).shape
print X_test.shape,y_test.shape

(105L,)
(45L, 80L) (45, 1)


In [421]:
X=LogReg.predict(X_train)
y=y_train.reset_index()['yellow']
X=pd.concat([pd.DataFrame(X),pd.DataFrame(y)],axis=1, ignore_index=True)
X['count']=1
Acc=X.pivot_table(index=0,columns=1,values='count',aggfunc=np.sum)
rate=float(Acc[0][0]+Acc[1][1])/Acc.sum().sum()
rate

0.8

In [403]:
X=LogReg.predict(X_test)
y=y_test.reset_index()['yellow']
X=pd.concat([pd.DataFrame(X),y],axis=1, ignore_index=True)
X['count']=1
Acc=X.pivot_table(index=0,columns=1,values='count',aggfunc=np.sum)
Acc=Acc.replace("NaN",0)
rate=float(Acc[0][0]+Acc[1][1])/Acc.sum().sum()
rate

0.6444444444444445

## Bayes

In [404]:
from sklearn.naive_bayes import GaussianNB
NB=GaussianNB()
NB.fit(X_train, y_train)

C:\Users\junior\Anaconda2\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(priors=None)

In [405]:
X=NB.predict(X_train)
y=y_train.reset_index()['yellow']
X=pd.concat([pd.DataFrame(X),pd.DataFrame(y)],axis=1, ignore_index=True)
X['count']=1
Acc=X.pivot_table(index=0,columns=1,values='count',aggfunc=np.sum)
rate=float(Acc[0][0]+Acc[1][1])/Acc.sum().sum()
print rate

0.87619047619


In [406]:
X=NB.predict(X_test)
y=y_test.reset_index()['yellow']
X=pd.concat([pd.DataFrame(X),pd.DataFrame(y)],axis=1, ignore_index=True)
X['count']=1
Acc=X.pivot_table(index=0,columns=1,values='count',aggfunc=np.sum)
rate=float(Acc[0][0]+Acc[1][1])/Acc.sum().sum()
print rate

0.511111111111
